In [1]:
%cd /Users/rurikoimai/Documents/GitHub/Satellite_Pool_Detection

/Users/rurikoimai/Documents/GitHub/Satellite_Pool_Detection


In [2]:
from MinimumBoundingBox import MinimumBoundingBox
import xml.etree.cElementTree as ET
import json

In [3]:
# Opening JSON file
f = open('./metadata.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)

# Closing file
f.close()
# data = {"42.7363609_2.8868181.png": {"has_pool":True, 
#                                      "center": {"lat": 42.7363609, "lon": 2.8868181}, 
#                                      "bounds_lat_lon": [
#                                          {"lat": 42.7364046, "lon": 2.8867693}, 
#                                          {"lat": 42.7363591, "lon": 2.8867414},
#                                          {"lat": 42.7363172, "lon": 2.8868668}, 
#                                          {"lat": 42.7363628, "lon": 2.8868948}, 
#                                          {"lat": 42.7364046, "lon": 2.8867693}], 
#                                      "bounds_x_y": [
#                                          {"x": 219.88854424638464, "y": 197.97246403701524}, 
#                                          {"x": 199.24285541121722, "y": 243.81349049302645}, 
#                                          {"x": 292.03745689232903, "y": 286.02750490829436}, 
#                                          {"x": 312.75714461835867, "y": 240.08575994312486}, 
#                                          {"x": 219.88854424638464, "y": 197.97246403701524}]},
#        }

In [4]:
#want it in this format
# <?xml version="1.0"?>
# <annotation>
#     <filename>000000019.jpg</filename>
#     <source>
#         <database>Unknown</database>
#     </source>
#     <size>
#         <width>224</width>
#         <height>224</height>
#         <depth>3</depth>
#     </size>
#     <object>
#         <name>pool</name>
#         <bndbox>
#             <xmin>168</xmin>
#             <ymin>0</ymin>
#             <xmax>212</xmax>
#             <ymax>20</ymax>
#         </bndbox>
#     </object>
# </annotation>

In [5]:
f_names = [file for file in data.keys()]

In [6]:
len(f_names)

14964

In [7]:
!mkdir labels/
%cd labels/

/Users/rurikoimai/Documents/GitHub/Satellite_Pool_Detection/labels


In [8]:
# for idx, f_name in enumerate(f_names):
for f_name in f_names:
    # find coordinates
    coords = [coord for coord in data[f_name]['bounds_x_y']]
    
    points = []
    i = 0
    while i < len(coords):
        points.append((coords[i]['x'], coords[i]['y']))
        i += 1
    
    # convert polygon to a box
    bounding_box = MinimumBoundingBox(points)
    coors = bounding_box.corner_points
    
    # find min, max coordinates of the box
    max_xy = max(coors, key=lambda x: (x[0], x[1]))
    min_xy = min(coors, key=lambda x: (x[0], x[1]))
    
    
    # write to XML file
    annot = ET.Element("annotation")
    ET.SubElement(annot, "filename").text = f_name
    size = ET.SubElement(annot, "size")
    ET.SubElement(size, "width").text = "512"
    ET.SubElement(size, "height").text = "512"
    ET.SubElement(size, "depth").text = "3"
    object_ = ET.SubElement(annot, "object")
    ET.SubElement(object_, "name").text = "pool"
    bndbox = ET.SubElement(object_, "bndbox")
    ET.SubElement(bndbox, "xmin").text = str(int(float(min_xy[0])))
    ET.SubElement(bndbox, "ymin").text = str(int(float(min_xy[1])))
    ET.SubElement(bndbox, "xmax").text = str(int(float(max_xy[0])))
    ET.SubElement(bndbox, "ymax").text = str(int(float(max_xy[1])))

    # file name w/o .png
    file = f_name[:-4]
    tree = ET.ElementTree(annot)
    tree.write(f"{file}.xml")